<img src="https://raw.githubusercontent.com/FUlyankin/Parsers/master/images%20/cian_main_theme.png" height="1000" width="1000"> 

<img align="center" src="https://www.cian.ru/promo/adv/assets/images/logo.svg" height="300" width="300"> 


# <center> Грабёж, разбой и другие незаконные деяния с помощью Python. <br> <br> Продолжаем ограбление. Задействуем союзника - Google!  </center>
---------

## 1. Что такое API?

API (Application Programming Interface) это уже готовый код, который можно всунуть в свой код!  Многие разработчики, в том числе Google и Вконтакте представляют свои уже готовые решения для вашей разработки. 

Обычно обращение к API происходит точно также как к обычной ссылке через пакет `requests`.

In [1]:
import pandas as pd
import numpy as np

import requests

Обратиться через этот пакет обычно нужно к простой банальной ссылке, в которую будет вшито то, что вы хотели бы для себя достать. Итак, заходим в [документацию по API для Google maps](https://developers.google.com/maps/documentation/webservices/), которая была заботливо для нас написана и начинаем её использовать.



Мы хотим получить расстояние, которое нужно пройти пешком или проехать на машине от каждой квартиры до ближайшего метро. Для этого мы будем использовать метод, который называется *distancematrix*. 

*Google Distance Matrix API* - это сервис, который предоставляет расстояние и время пути между указанными точками. Ссылка для запроса к API будет иметь следующую структуру

> https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial&origins=Washington,DC&destinations=New+York+City,NY&key=YOUR_API_KEY

В первой строке указано название сервиса, к API которого мы обращаемся, метод который мы используем ,*distancematrix*, формат, в котором мы хотели бы получить ответ, *json*, и объекты, между которыми мы ищем расстояние, *Origins* - начало отсчета, *Destinations* - конец отсчета. Кроме того через амперсанды в ссылку можно добавить кучу других параметров. Например: *Mode* - способ перемещения, *Language* - язык, на котором мы хотели бы получить ответ на наш запрос.

Пробуем!

In [2]:
mainpage = "http://maps.googleapis.com/maps/api/distancematrix/json?"

place_one = "м. Юго-Западная, Москва"
place_two = "ул. Академика Анохина 20, Москва"
sposob = "walking"
YOU_API_KEY = ''

parameters = 'origins='+place_one+'&destinations='+place_two+'&mode='+sposob+'&language=ru-Ru'

itog_url = mainpage + parameters
itog_url

'http://maps.googleapis.com/maps/api/distancematrix/json?origins=м. Юго-Западная, Москва&destinations=ул. Академика Анохина 20, Москва&mode=walking&language=ru-Ru'

In [3]:
response = requests.get(itog_url)
response

<Response [200]>

Ура! Благосоавенный 200-всё-очень-хорошо-ответ! Попробуем сделать как мы делали раньше и посмотреть html нашей странички.

In [4]:
content = response.content
content

b'{\n   "destination_addresses" : [\n      "\xd1\x83\xd0\xbb. \xd0\x90\xd0\xbd\xd0\xbe\xd1\x85\xd0\xb8\xd0\xbd\xd0\xb0 \xd0\x90\xd0\xba\xd0\xb0\xd0\xb4\xd0\xb5\xd0\xbc\xd0\xb8\xd0\xba\xd0\xb0, 20, \xd0\x9c\xd0\xbe\xd1\x81\xd0\xba\xd0\xb2\xd0\xb0, \xd0\xa0\xd0\xbe\xd1\x81\xd1\x81\xd0\xb8\xd1\x8f, 119602"\n   ],\n   "origin_addresses" : [ "\xd0\xae\xd0\xb3\xd0\xbe-\xd0\x97\xd0\xb0\xd0\xbf\xd0\xb0\xd0\xb4\xd0\xbd\xd0\xb0\xd1\x8f, \xd0\x9c\xd0\xbe\xd1\x81\xd0\xba\xd0\xb2\xd0\xb0, \xd0\xa0\xd0\xbe\xd1\x81\xd1\x81\xd0\xb8\xd1\x8f, 119526" ],\n   "rows" : [\n      {\n         "elements" : [\n            {\n               "distance" : {\n                  "text" : "0,9 \xd0\xba\xd0\xbc",\n                  "value" : 898\n               },\n               "duration" : {\n                  "text" : "11 \xd0\xbc\xd0\xb8\xd0\xbd.",\n                  "value" : 687\n               },\n               "status" : "OK"\n            }\n         ]\n      }\n   ],\n   "status" : "OK"\n}\n'

Ух! Какая страшная штука. Но тем не менее она похожа именно на то, что нам нужно. Как я это понял? Интуиция... Но почему тогда она такая некрасивая? Всё дело в том, что мы пытаемся применить к нашему контенту те же самые методы, что мы использовали для html разметки. Тем не менее данные были скачены в формате json. Если бы мы указали в ссылке для скачки lxml-формат, то мы могли бы даже поработать с нашими данными красивым супом. 

Тем не менее, мы указали другой формат, потому что он более удобный. Смотрите!

In [5]:
content = response.json()
content

{'destination_addresses': ['ул. Анохина Академика, 20, Москва, Россия, 119602'],
 'origin_addresses': ['Юго-Западная, Москва, Россия, 119526'],
 'rows': [{'elements': [{'distance': {'text': '0,9 км', 'value': 898},
     'duration': {'text': '11 мин.', 'value': 687},
     'status': 'OK'}]}],
 'status': 'OK'}

Секундочку... Это что? Словарь? Да! Это словарь! 

In [6]:
type(content)

dict

Вытаскиваем из этого словаря всё, что нам необходимо. 

In [7]:
content['destination_addresses']

['ул. Анохина Академика, 20, Москва, Россия, 119602']

In [8]:
content['rows'][0]['elements'][0]['duration']['text']

'11 мин.'

Точно также можно качать расстояние между объектами по координатам! Подгружаем наш ЦИАНовский датасет!

In [9]:
df = pd.read_csv('clean_data_for_work.csv', header=0, sep=',' ,index_col=0)
longtitude = list(df['longitude'])
latitude = list(df['latitude'])
metro = list(df['metro_name'])

In [10]:
def infa_downloader(place_one,place_two,sposob):
    
    # создлали ссылку по входным параметрам
    mainpage = "http://maps.googleapis.com/maps/api/distancematrix/json?"
    parameters = 'origins='+place_one+'&destinations='+place_two+'&mode='+sposob+'&language=ru-Ru'+'&key='
    itog_url = mainpage + parameters
    
    # сделали запрос и вытащили инфу
    response = requests.get(itog_url)
    content = response.json()
    return(content['rows'][0]['elements'][0]['duration']['text'])

In [11]:
str(longtitude[0])+','+str(latitude[0])

'37.544467,55.860792'

In [12]:
metro[0]

'Петровско-Разумовская,'

In [13]:
infa_downloader(str(latitude[1001])+','+str(longtitude[1001]),metro[1001],"walking")

'1 час. 26 мин.'

In [19]:
infa_downloader(str(latitude[1001])+','+str(longtitude[1001]),metro[1001],"driving")

'35 мин.'

Вроде бы всё хорошо. Дело осталось за малым - написать цикл! 

# НЕТ! 

Ничего подобного. Сначала нужно изучить ограничения.

<img src="https://raw.githubusercontent.com/FUlyankin/Parsers/master/images%20/Workspace%201_180.png" height="700" width="700">

Неожиданно выясняется, что мы можем делать в течение суток не более 2500 запросов. Чёрт возьми! Какая боль, какая обида и какие страдания... Ограничения выяснили, попробуем теперь написать цикл. Авось прокатит...


In [ ]:
walk_dist  = [ ]
drive_dist = [ ]

for i in range(len(metro)):
    walk_dist.append( infa_downloader(str(latitude[i])+','+str(longtitude[i]),metro[i],"walking") )
    drive_dist.append( infa_downloader(str(latitude[i])+','+str(longtitude[i]),metro[i],"driving") )

Неожиданно вспоминаем, что такой цикл не заработает, потому что метро есть не везде... Переписываем цикл...

In [20]:
walk_dist  = [ ]
drive_dist = [ ]

for i in range(len(metro[:100])):
    try:
        walk_dist.append( infa_downloader(str(latitude[i])+','+str(longtitude[i]),metro[i],"walking") )
    except Exception:
        walk_dist.append(np.nan)
    
    try:
        drive_dist.append( infa_downloader(str(latitude[i])+','+str(longtitude[i]),metro[i],"driving") )
    except Exception:
        drive_dist.append(np.nan)    
        
    # Мы хотим видеть где именно находится наш цикл!
    if i%10 == 0:
        print(i,drive_dist[i],walk_dist[i])

0 10 мин. 51 мин.
10 32 мин. 7 ч. 45 мин.
20 10 ч. 50 мин. 7 дн. 7 ч.
30 5 мин. 20 мин.
40 6 мин. 8 мин.
50 3 мин. 9 мин.
60 7 мин. 10 мин.
70 2 мин. 10 мин.
80 42 мин. 7 ч. 24 мин.
90 17 мин. 1 час. 31 мин.


In [24]:
print(drive_dist,walk_dist,sep='\n \n \n')

['10 мин.', '7 мин.', '5 мин.', '36 мин.', '10 ч. 48 мин.', '34 мин.', '38 мин.', '8 мин.', '3 мин.', '32 мин.', '32 мин.', '32 мин.', nan, nan, '32 мин.', nan, '32 мин.', nan, '10 ч. 48 мин.', '30 мин.', '10 ч. 50 мин.', '35 мин.', '3 мин.', '5 мин.', '4 мин.', nan, '30 мин.', '17 мин.', '10 мин.', '5 мин.', '5 мин.', '30 мин.', '21 мин.', '30 мин.', '24 мин.', '8 мин.', '7 мин.', '32 мин.', '30 мин.', '30 мин.', '6 мин.', '5 мин.', '25 мин.', '7 мин.', '23 мин.', '29 мин.', '3 мин.', '30 мин.', nan, '30 мин.', '3 мин.', nan, '30 мин.', '7 мин.', '33 мин.', nan, '3 мин.', '8 мин.', '7 мин.', '7 мин.', '7 мин.', '23 мин.', '30 мин.', '17 мин.', '12 мин.', '6 мин.', '35 мин.', '32 мин.', '32 мин.', '17 мин.', '2 мин.', '17 мин.', '9 мин.', '31 мин.', '9 мин.', nan, '9 мин.', nan, nan, nan, '42 мин.', '14 мин.', '17 мин.', '17 мин.', '5 мин.', '5 мин.', '5 мин.', '17 мин.', '17 мин.', '9 мин.', '17 мин.', '23 мин.', '17 мин.', '1 мин.', '17 мин.', nan, '32 мин.', '17 мин.', '30 мин.', '2

Цикл работает довольно быстро и обрубается как только число отведённых нам запросов заканчивается. Вместо хороших ответов на наши запросы начинают появляться вот такие: 

``` 
{'destination_addresses': [],
 'error_message': 'You have exceeded your daily request quota for this API. We recommend registering for a key at the Google Developers Console: https://console.developers.google.com/apis/credentials?project=_',
 'origin_addresses': [],
 'rows': [],
 'status': 'OVER_QUERY_LIMIT'}

```


Что нам с этим делать? 

* Выход первый: качать несколько дней, качать с разных компов
* Выход второй: обмануть гугл, подключив кусок кода, который динамически меняет наш IP

Но об этом чуть позже...  Когда мы станем совсем продвинутыми юзерами...